In [4]:
import pyaudio
import math
import numpy as np

from lib.test import *

In [5]:
audioport = pyaudio.PyAudio()

In [80]:
framerate = 44100
latency = (1024 + 512 - 256 - 128 + 64 + 32 - 16 + 8 + 4 - 2 + 1)/framerate
latency

0.0281859410430839

In [81]:
class SineGen():
    def __init__(self, freq, amp=1):
        self.freq = freq
        self.amp = amp
        self.phase = 0
        self.sine_gen = self.sine_frame()
        self.stream = audioport.open(
            rate=framerate, 
            channels=1, 
            format=pyaudio.paFloat32, 
            output=True, 
            stream_callback=self.get_frame)
    
    def get_frame(self, in_data, frame_count, time_info, status):
        frame = []
        for f in range(frame_count):
            # block until within latency
            while self.stream.get_time() < time_info['output_buffer_dac_time'] + f / framerate - latency:
                pass
            frame.append(next(self.sine_gen))
        return (np.array(frame, dtype=np.float32).tobytes(), pyaudio.paContinue)
        
    def sine_frame(self):
        while True:
            yield self.amp * math.sin(self.phase)
            self.phase += 2 * math.pi * self.freq / framerate

In [82]:
sine = SineGen(440, amp=0.8)

In [83]:
sine.stream.stop_stream()

In [86]:
sine = SineGen(330, amp=0.1)

In [87]:
sine.stream.stop_stream()